In [0]:
import logging
from src.data_requests import DataRequests
from src.data_transformers import DataTransformers
from src.save_data import SaveData


def job(logger):
    buffers = []
    for mes in ["01", "02", "03", "04", "05"]:
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-{mes}.parquet"
        requisicao = DataRequests(url=url, logger=logger)
        buffers.append(requisicao.get_response())

    transformacao = DataTransformers(
        logger=logger, spark=spark, responses=buffers, partition="tpep_pickup_datetime"
    )

    df_bronze = transformacao.create_dataframe_spark()

    save_bronze = SaveData(
        logger=logger,
        spark=spark,
        df=df_bronze,
        layer="bronze",
        table="yellow_trip",
        mode_save="overwrite",
        format_save="delta",
        partition="partition",
    )

    save_bronze.saveData()


if __name__ == "__main__":
    logger = logging.getLogger("silver")
    logging.basicConfig(
        level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s - %(message)s"
    )

    job(logger)